In [1]:
#!pip install pymupdf
#!pip install langchain
#!pip install lancedb
#!pip install -U langchain_ollama

In [1]:
import fitz
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import LanceDB
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM

In [3]:
import lancedb

/home/u/RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
doc = fitz.open("DeepSeek.pdf")

In [5]:
texts, metadatas = [], []
db = lancedb.connect("./lancedb")
embeddings = OllamaEmbeddings(model="deepseek-r1:14b")

In [6]:
llm = OllamaLLM(model="deepseek-r1:7b")

In [ ]:
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    texts.append(page.get_text())
    metadatas.append(doc.metadata)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document=text_splitter.create_documents(texts,metadatas=metadatas)
vectorstore = LanceDB.from_documents(document, embeddings, connection=db)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [ ]:
query = "What are the main topics discussed in the documents?"

In [ ]:
result = qa.invoke(query)
print(result)